In [7]:
import boto3
from pydub import AudioSegment
from pydub.playback import play
from pydub.effects import normalize
from io import BytesIO

def add_reverb(audio: AudioSegment) -> AudioSegment:
    # Extend audio by adding silence to allow the echo to fade
    silence_duration = 250  # 250 ms of silence
    silence = AudioSegment.silent(duration=silence_duration)
    extended_audio = audio + silence  # Append silence to the end

    # Simulated reverb effect: layering with a delayed, quieter copy
    delay = extended_audio - 15  # Lower volume for the delayed layer
    combined = extended_audio.overlay(delay, position=250)  # 250ms delay
    return combined

# Initialize Polly
polly = boto3.client("polly")

# Synthesize speech
response = polly.synthesize_speech(
    Text="Hello, this is a reverb test.",
    OutputFormat="mp3",
    VoiceId="Salli"
)

# Read audio stream into a seekable BytesIO object
audio_stream = BytesIO(response["AudioStream"].read())

# Load the MP3 data into a pydub AudioSegment
audio = AudioSegment.from_file(audio_stream, format="mp3")

# Add effects: Normalize and apply reverb
audio = normalize(audio)  # Normalize volume
audio_with_reverb = add_reverb(audio)


# Play the processed audio
play(audio_with_reverb)

In [2]:
# Play the processed audio
play(audio_with_reverb)

In [15]:
import threading

# Pan the audio to the right ear
right_ear_audio = audio_with_reverb.pan(1.0)

# Pan the audio to the left ear
left_ear_audio = audio_with_reverb.pan(-1.0)

# Create a silent audio segment of 0.5 seconds
silence = AudioSegment.silent(duration=500)

# Concatenate the right ear audio with the silence and then the left ear audio
combined_audio = right_ear_audio.overlay(left_ear_audio, position=1000)
# Function to play audio
def play_audio(audio_segment):
    play(audio_segment)

# Create threads for playing audio
thread1 = threading.Thread(target=play_audio, args=(combined_audio,))
thread2 = threading.Thread(target=play_audio, args=(combined_audio,))

# Start the threads
thread1.start()
thread2.start()

# Wait for both threads to finish
thread1.join()
thread2.join()
# Play the combined audio
play(combined_audio)